## importing Library and environment variables

In [14]:
from __future__ import annotations

import operator
import os
import re
from datetime import date, timedelta
from pathlib import Path
from typing import TypedDict, List, Optional, Literal, Annotated

from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.types import Send

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()


True

## Pydantic Schemas

In [15]:
#-----------------------------
# 1) Schemas
#----------------------------
class Task(BaseModel):
    id: int
    title: str
    goal: str = Field(
        ...,
        description = "One sentence describing what the reader should be able to do/understand after this section.",
    )
    bullets: List[str] = Field(
        ...,
        min_length=3,
        max_length=6,
        description="3–6 concrete, non-overlapping subpoints to cover in this section.",
    )
    target_words: int = Field(..., description="Target word count for this section (120–550).")
    
    tags: List[str] = Field(default_factory=list)
    requires_research: bool = False
    requires_citations: bool = False
    requires_code: bool = False
    

class Plan(BaseModel):
    blog_title: str
    audience: str
    tone: str
    blog_kind: Literal["explainer", "tutorial", "news_roundup", "comparison", "system_design"] = "explainer"
    constraints: List[str] = Field(default_factory=list)
    tasks: List[Task]

class EvidenceItem(BaseModel):
    title: str
    url: str
    published_at: Optional[str] = None  # keep if Tavily provides; DO NOT rely on it
    snippet: Optional[str] = None
    source: Optional[str] = None
    

## LangGraph State Schema

In [21]:
class State(TypedDict):
    topic: str
    
    #routing / research
    mode: str
    need_research: bool
    queries: List[str]
    evidence: List[EvidenceItem]
    plan: Optional[Plan]
    
    #workers
    sections: Annotated[List[tuple[int,str]], operator.add] # (task_id, section_md)
    
    #reducer/image
    merged_md: str
    md_with_placeholders: str
    image_specs: List[dict]
    final: str
    

    

## OpenAI Client

In [19]:
llm = ChatOpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url= "https://generativelanguage.googleapis.com/v1beta/openai/"
)

## LangGraph Graph Architecture

In [23]:
#----------------------
# build main graph
#-----------------------
g = StateGraph(State)

#added nodes
g.add_node("router", router_node)
g.add_node("research", research_node)
g.add_node("orchestrator", orchestrator_node)
g.add_node("worker", worker_node)
g.add_node("reducer", reducer_subgraph)

# graph construction by joining edges
g.add_edge(START, "router")
g.add_conditional_edges("router", route_next, {"research":"research","orchestrator":"orchestrator"})
g.add_edge("research", "orchestrator")
g.add_conditional_edges("orchestrator", fanout, ["worker"])
g.add_edge("worker", "reducer")
g.add_edge("reducer",END)

app = g.compile()
app


NameError: name 'research_node' is not defined

In [22]:
def router_node(state: State) -> dict:
    pass